In [ ]:
'''Create Figure 3, showing the bias due to the different parameters.'''

'''Bias Interaction Plots With 100 realizations of the
synthetic transition for each combination of parameters.'''

import numpy as np
import pandas as pd
import proplot as pplt

#Decadal Data
sigma = np.empty(10000)
tau = np.empty(10000)
GS_slope = np.empty(10000)
GIS_slope = np.empty(10000)
duration = np.empty(10000)

for i in range(10000):
    sigma[i] = 0.02 + 0.02 * (i%10)
    tau[i] = 3.0 + 3.0 * int((i%100)/10)
    GS_slope[i] = -1.8e-3 + 4e-4 * int((i%100)/10)
    GIS_slope[i] = 0.0 - 3e-4 * int((i%100)/10)
    duration[i] = 10.0 + 10.0 * int((i%100)/10)

df = pd.read_pickle('Data/decadal_synthetics/double_GS_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'GS_slope': GS_slope}
double_GS_df = pd.DataFrame(data=d)

df = pd.read_pickle('Data/decadal_synthetics/double_GIS_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'GIS_slope': GIS_slope}
double_GIS_df = pd.DataFrame(data=d)

df = pd.read_pickle('Data/decadal_synthetics/double_dur_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'duration': duration}
double_dur_df = pd.DataFrame(data=d)

df = pd.read_pickle('Data/decadal_synthetics/double_tau_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'tau': tau}
double_tau_df = pd.DataFrame(data=d)


fig = pplt.figure(refwidth=4, share=False)
axs = fig.add_subplots(ncols=4, nrows=2)
fig.format(abc=True, abcloc='ul', fontsize=18)
axs.format(fontsize=18)

double_GIS_df['sig_GIS'] = double_GIS_df['sigma'] + double_GIS_df['GIS_slope']
double_GS_df['sig_GS'] = double_GS_df['sigma'] + double_GS_df['GS_slope']
double_dur_df['sig_dur'] = double_dur_df['sigma'] + double_dur_df['duration']
double_tau_df['sig_tau'] = double_tau_df['sigma'] + double_tau_df['tau']

sigma_GIS_mean = np.empty((10,10))

for i in range(100):
    sigma_GIS_mean[i%10,int(i/10)] = double_GIS_df.groupby('sig_GIS').mean()['time'].values[i]

sigma_GS_mean = np.empty((10,10))

for i in range(100):
    sigma_GS_mean[i%10,int(i/10)] = double_GS_df.groupby('sig_GS').mean()['time'].values[i]

sigma_dur_mean = np.empty((10,10))

for i in range(100):
    sigma_dur_mean[i%10,int(i/10)] = double_dur_df.groupby('sig_dur').mean()['time'].values[i]

sigma_tau_mean = np.empty((10,10))

for i in range(100):
    sigma_tau_mean[i%10,int(i/10)] = double_tau_df.groupby('sig_tau').mean()['time'].values[i]

contour_levs = [-11,-7,-4,-2,-1,0,1,2,4,7,11]

cmap1 = pplt.Colormap('RdBu_r')

axs[4].format(xlabel='Noise / Signal', ylabel='Autocorrelation Time / Years',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=tau[np.arange(10,110,step=20)], 
             yminorticks=tau[np.arange(100,step=20)])
dur = axs[4].contourf(x=sigma[:10], y=tau[np.arange(100,step=10)], cmap=cmap1,
                       z=np.transpose(sigma_tau_mean),levels=contour_levs, extend='both')

axs[5].format(xlabel='Noise / Signal', ylabel='Greenland Stadial Slope / $Kiloyears^{-1}$',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=1000*GS_slope[np.arange(10,110,step=20)], 
              yminorlocator=1000*GS_slope[np.arange(100,step=20)])
GS = axs[5].contourf(x=sigma[:10], y=1000*GS_slope[np.arange(100,step=10)], cmap=cmap1,
                       z=sigma_GS_mean,levels=contour_levs, extend='both')

axs[6].format(xlabel='Noise / Signal', ylabel='Absolute Greenland Interstadial Slope \n / $Kiloyears^{-1}$',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=np.abs(1000*GIS_slope[np.arange(10,110,step=20)]),
              yminorlocator=0.3)
slope = axs[6].contourf(x=sigma[:10], y=np.abs(1000*GIS_slope[np.arange(100,step=10)]), cmap=cmap1,
                       z=np.flip(sigma_GIS_mean,axis=0),levels=contour_levs, extend='both')

axs[7].format(xlabel='Noise / Signal', ylabel='Transition Duration / Years',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=duration[np.arange(10,110,step=20)], yminorlocator=10)
dur = axs[7].contourf(x=sigma[:10], y=duration[np.arange(100,step=10)], cmap=cmap1,
                       z=np.transpose(sigma_dur_mean),levels=contour_levs, extend='both')


colorbar = fig.colorbar(slope, loc='bottom')
colorbar.ax.tick_params(labelsize=18)
colorbar.ax.set_title('Bias / Years', fontsize=18)

#Annual data

for i in range(10000):
    sigma[i] = 0.04 + 0.04 * (i%10)
    tau[i] = 0.4 + 0.4 * int((i%100)/10)
    GS_slope[i] = -1.8e-3 + 4e-4 * int((i%100)/10)
    GIS_slope[i] = 0.0 - 3e-4 * int((i%100)/10)
    duration[i] = 10.0 + 10.0 * int((i%100)/10)

df = pd.read_pickle('Data/annual_synthetics/double_GS_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'GS_slope': GS_slope}
double_GS_df = pd.DataFrame(data=d)

df = pd.read_pickle('Data/annual_synthetics/double_GIS_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'GIS_slope': GIS_slope}
double_GIS_df = pd.DataFrame(data=d)

df = pd.read_pickle('Data/annual_synthetics/double_dur_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'duration': duration}
double_dur_df = pd.DataFrame(data=d)

df = pd.read_pickle('Data/annual_synthetics/double_tau_pickle')
df = df.where(df['time']>300,other=np.nan)
df = df.where(df['time']<450,other=np.nan)
time = df['time'] - 400
d = {'time': time, 'sigma': sigma, 'tau': tau}
double_tau_df = pd.DataFrame(data=d)

double_GIS_df['sig_GIS'] = double_GIS_df['sigma'] + double_GIS_df['GIS_slope']
double_GS_df['sig_GS'] = double_GS_df['sigma'] + double_GS_df['GS_slope']
double_dur_df['sig_dur'] = double_dur_df['sigma'] + double_dur_df['duration']
double_tau_df['sig_tau'] = double_tau_df['sigma'] + 10*double_tau_df['tau']

sigma_GIS_mean = np.empty((10,10))

for i in range(100):
    sigma_GIS_mean[i%10,int(i/10)] = double_GIS_df.groupby('sig_GIS').mean()['time'].values[i]

sigma_GS_mean = np.empty((10,10))

for i in range(100):
    sigma_GS_mean[i%10,int(i/10)] = double_GS_df.groupby('sig_GS').mean()['time'].values[i]

sigma_dur_mean = np.empty((10,10))

for i in range(100):
    sigma_dur_mean[i%10,int(i/10)] = double_dur_df.groupby('sig_dur').mean()['time'].values[i]

sigma_tau_mean = np.empty((10,10))

for i in range(100):
    sigma_tau_mean[i%10,int(i/10)] = double_tau_df.groupby('sig_tau').mean()['time'].values[i]

cmap1 = pplt.Colormap('RdBu_r')

axs[0].format(xlabel='Noise / Signal', ylabel='Autocorrelation Time / Years',
              xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=tau[np.arange(10,110,step=20)], 
              yminorticks=tau[np.arange(100,step=20)])
dur = axs[0].contourf(x=sigma[:10], y=tau[np.arange(100,step=10)], cmap=cmap1,
                       z=np.transpose(sigma_tau_mean),levels=contour_levs, extend='both')

axs[1].format(xlabel='Noise / Signal', ylabel='Greenland Stadial Slope / $Kiloyears^{-1}$',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=1000*GS_slope[np.arange(10,110,step=20)], 
              yminorlocator=1000*GS_slope[np.arange(100,step=20)])
GS = axs[1].contourf(x=sigma[:10], y=1000*GS_slope[np.arange(100,step=10)], cmap=cmap1,
                       z=sigma_GS_mean,levels=contour_levs, extend='both')

axs[2].format(xlabel='Noise / Signal', ylabel='Absolute Greenland Interstadial Slope \n / $Kiloyears^{-1}$',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=np.abs(1000*GIS_slope[np.arange(10,110,step=20)]),
              yminorlocator=0.3)
slope = axs[2].contourf(x=sigma[:10], y=np.abs(1000*GIS_slope[np.arange(100,step=10)]), cmap=cmap1,
                       z=np.flip(sigma_GIS_mean,axis=0),levels=contour_levs, extend='both')

axs[3].format(xlabel='Noise / Signal', ylabel='Transition Duration / Years',
             xticks=sigma[np.arange(1,11,step=2)], xminorlocator=0.02, yticks=duration[np.arange(10,110,step=20)], yminorlocator=10)
dur = axs[3].contourf(x=sigma[:10], y=duration[np.arange(100,step=10)], cmap=cmap1,
                       z=np.transpose(sigma_dur_mean),levels=contour_levs, extend='both')

# decadal: sigma, tau, GS slope, GIS slope, duration
tas = (0.06633234778696487, 13.649751415738818, 0.00016431508734689263, -0.0009404944834246113, 56.32925780771254)
pre = (0.1542468176758728, 7.610589558843324, 0.0003347760370749247, -0.0012734065625610814, 69.84867995820262)
ice = (0.06772528109764206, 13.522050331034816, 0.0001679453269451725, -0.0009241650626852406, 58.975578975895765)
AMOC = (0.09302892310996252, 20.317394958495132, 0.00028905757180442527, -0.0016966433176966102, 62.129599258809385)
NAO = (0.07601909051802441, 6.768150630222373, 0.0001115667911889947, -0.0007890345342570018, 58.206677821434035)

var_sigma = (tas[0],pre[0],ice[0],AMOC[0],NAO[0])
var_tau = (tas[1],pre[1],ice[1],AMOC[1],NAO[1])
var_GS_slope = 1000* np.array((tas[2],pre[2],ice[2],AMOC[2],NAO[2]))
var_GIS_slope = np.abs(1000* np.array((tas[3],pre[3],ice[3],AMOC[3],NAO[3])))
var_duration = (tas[4],pre[4],ice[4],AMOC[4],NAO[4])
markers = ('o', 'v', 's', '^', '*')
labels=('Temperature', 'Precipitation', 'Sea Ice', 'AMOC', 'NAO')

for i in range(5):
    axs[4].scatter(var_sigma[i], var_tau[i], edgecolor='white', color='black', s=200, marker=markers[i], label=labels[i])
    axs[5].scatter(var_sigma[i], var_GS_slope[i], edgecolor='white', color='black', s=200, marker=markers[i])
    axs[6].scatter(var_sigma[i], var_GIS_slope[i], edgecolor='white', color='black', s=200, marker=markers[i])
    axs[7].scatter(var_sigma[i], var_duration[i], edgecolor='white', color='black', s=200, marker=markers[i])


fig.legend(loc='bottom',fontsize=18,ncols=5, mode='expand')